# 03 - TFX Interactive Training Pipeline Execution

The purpose of this notebook is to interactively run the following `TFX` pipeline steps:
1. Receive hyperparameters using `hyperparam_gen` custom Python component.
2. Extract data from BigQuery using `BigQueryExampleGen` component.
3. Validate the raw data using `StatisticsGen` and `ExampleValidator` components.
4. Process the data using `Transform` component.
5. Train a custom model using `Trainer` component.
7. Evaluate and Validate the custom model using `ModelEvaluator` component.
7. Save the blessed to model registry location using `Pusher` component.
8. Upload the model to Vertex AI using `vertex_model_pusher` custom Python component

The custom components are implemented in the [tfx_pipeline/components.py](tfx_pipeline/components) module.

## Setup

### Import libraries

In [1]:
import os
import json
import numpy as np
import tfx as tfx
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
from tensorflow_transform.tf_metadata import schema_utils
import logging

from src.common import features
from src.model_training import data
from src.tfx_pipelines import components

logging.getLogger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')

print("TFX Version:", tfx.__version__)
print("Tensorflow Version:", tf.__version__)

2022-09-10 23:40:02.104176: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-10 23:40:02.104226: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TFX Version: 1.8.0
Tensorflow Version: 2.8.2


### Setup Google Cloud project

In [2]:
PROJECT = 'mwpmltr' # Change to your project id.
REGION = 'us-central1' # Change to your region.
BUCKET = 'gcp-certification-chicago-taxi-demo' # Change to your bucket name.
SERVICE_ACCOUNT = "55590906972-compute@developer.gserviceaccount.com"

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP project id
    BUCKET = PROJECT
    # Try to create the bucket if it doesn't exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
PARENT = f"projects/{PROJECT}/locations/{REGION}"
    
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)
print("Service Account:", SERVICE_ACCOUNT)
print("Vertex API Parent URI:", PARENT)

Project ID: mwpmltr
Region: us-central1
Bucket name: gcp-certification-chicago-taxi-demo
Service Account: 55590906972-compute@developer.gserviceaccount.com
Vertex API Parent URI: projects/mwpmltr/locations/us-central1


### Set configurations

In [3]:
VERSION = 'v01'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'

WORKSPACE = f'gs://{BUCKET}/{DATASET_DISPLAY_NAME}'
RAW_SCHEMA_DIR = 'src/raw_schema'

MLMD_SQLLITE = 'mlmd.sqllite'
ARTIFACT_STORE = os.path.join(WORKSPACE, 'tfx_artifacts_interactive')
MODEL_REGISTRY = os.path.join(WORKSPACE, 'model_registry')
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME)

In [4]:
print(f"MODEL_DISPLAY_NAME= {MODEL_DISPLAY_NAME}")
print(f"WORKSPACE= {WORKSPACE}")
print(f"RAW_SCHEMA_DIR= {RAW_SCHEMA_DIR}")
print(f"MLMD_SQLLITE= {MLMD_SQLLITE}")
print(f"ARTIFACT_STORE= {ARTIFACT_STORE}")
print(f"MODEL_REGISTRY= {MODEL_REGISTRY}")
print(f"PIPELINE_NAME= {PIPELINE_NAME}")
print(f"PIPELINE_ROOT= {PIPELINE_ROOT}")

MODEL_DISPLAY_NAME= chicago-taxi-tips-classifier-v01
WORKSPACE= gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips
RAW_SCHEMA_DIR= src/raw_schema
MLMD_SQLLITE= mlmd.sqllite
ARTIFACT_STORE= gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/tfx_artifacts_interactive
MODEL_REGISTRY= gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/model_registry
PIPELINE_NAME= chicago-taxi-tips-classifier-v01-train-pipeline
PIPELINE_ROOT= gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/tfx_artifacts_interactive/chicago-taxi-tips-classifier-v01-train-pipeline


In [5]:
# Fix this error:
# ImportError: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.30' not found
LD_LIBRARY_PATH = f'/opt/conda/lib/'
os.environ["LD_LIBRARY_PATH"] = LD_LIBRARY_PATH

## Create Interactive Context

In [6]:
REMOVE_ARTIFACTS = True

if tf.io.gfile.exists(ARTIFACT_STORE) and REMOVE_ARTIFACTS:
    print("Removing previous artifacts...")
    tf.io.gfile.rmtree(ARTIFACT_STORE)
    
if tf.io.gfile.exists(MLMD_SQLLITE) and REMOVE_ARTIFACTS:
    print("Deleting previous mlmd.sqllite...")
    tf.io.gfile.rmtree(MLMD_SQLLITE)
    
print(f'Pipeline artifacts directory: {PIPELINE_ROOT}')
print(f'Local metadata SQLlit path: {MLMD_SQLLITE}')

Removing previous artifacts...
Deleting previous mlmd.sqllite...
Pipeline artifacts directory: gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/tfx_artifacts_interactive/chicago-taxi-tips-classifier-v01-train-pipeline
Local metadata SQLlit path: mlmd.sqllite


In [7]:
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = MLMD_SQLLITE
connection_config.sqlite.connection_mode = 3 # READWRITE_OPENCREATE
mlmd_store = mlmd.metadata_store.MetadataStore(connection_config)

context = InteractiveContext(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  metadata_connection_config=connection_config
)

## 1. Hyperparameter generation

In [8]:
hyperparams_gen = components.hyperparameters_gen(
    num_epochs=5,
    learning_rate=0.001,
    batch_size=512,
    hidden_units='64,64',
)

context.run(hyperparams_gen, enable_cache=True)

ExecutionResult(
    component_id: hyperparameters_gen
    execution_id: 1
    outputs:
        hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=hyperparameters_gen, output_key=hyperparameters, additional_properties={}, additional_custom_properties={}))

In [9]:
json.load(
    tf.io.gfile.GFile(
        os.path.join(
            hyperparams_gen.outputs['hyperparameters'].get()[0].uri, 'hyperparameters.json')
    )
)

{'num_epochs': 5,
 'batch_size': 512,
 'learning_rate': 0.001,
 'hidden_units': [64, 64]}

## 2. Data extraction

In [10]:
from src.common import datasource_utils
from tfx.extensions.google_cloud_big_query.example_gen.component import BigQueryExampleGen
from tfx.proto import example_gen_pb2, transform_pb2

### Extract train and eval splits

In [11]:
sql_query = datasource_utils.get_training_source_query(
    PROJECT, REGION, DATASET_DISPLAY_NAME, ml_use='UNASSIGNED', limit=5000)

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=4),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1),
        ]
    )
)

train_example_gen = BigQueryExampleGen(query=sql_query, output_config=output_config)

beam_pipeline_args=[
    f"--project={PROJECT}",
    f"--temp_location={os.path.join(WORKSPACE, 'tmp')}"
]

context.run(
    train_example_gen,
    beam_pipeline_args=beam_pipeline_args,
    enable_cache=True
)

/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:2471: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(


ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 2
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=BigQueryExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [12]:
context.run(
    train_example_gen,
    beam_pipeline_args=beam_pipeline_args,
    enable_cache=True
)

ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 3
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=BigQueryExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Extract test split

In [13]:
sql_query = datasource_utils.get_training_source_query(
    PROJECT, REGION, DATASET_DISPLAY_NAME, ml_use='TEST', limit=1000)

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="test", hash_buckets=1),
        ]
    )
)

test_example_gen = BigQueryExampleGen(query=sql_query, output_config=output_config)

beam_pipeline_args=[
    f"--project={PROJECT}",
    f"--temp_location={os.path.join(WORKSPACE, 'tmp')}"
]

context.run(
    test_example_gen,
    beam_pipeline_args=beam_pipeline_args,
    enable_cache=True
)

ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 4
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=BigQueryExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Read sample extract tfrecords

In [14]:
train_uri = os.path.join(train_example_gen.outputs['examples'].get()[0].uri, "Split-train/*")
source_raw_schema = tfdv.load_schema_text(os.path.join(RAW_SCHEMA_DIR, 'schema.pbtxt'))
raw_feature_spec = schema_utils.schema_as_feature_spec(source_raw_schema).feature_spec

def _parse_tf_example(tfrecord):
    return tf.io.parse_single_example(tfrecord, raw_feature_spec)

tfrecord_filenames = tf.data.Dataset.list_files(train_uri)
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
dataset = dataset.map(_parse_tf_example)

for raw_features in dataset.shuffle(1000).batch(3).take(1):
    for key in raw_features:
        print(f"{key}: {np.squeeze(raw_features[key], -1)}")
    print("")

2022-09-10 23:40:53.980955: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/
2022-09-10 23:40:53.980996: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-10 23:40:53.981018: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (python-20220624-164215-bjacob): /proc/driver/nvidia/version does not exist
2022-09-10 23:40:53.981378: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


dropoff_grid: [b'POINT(-87.7 42)' b'POINT(-87.6 41.9)' b'POINT(-87.9 42)']
euclidean: [21368.113  4884.611 21674.977]
loc_cross: [b'POINT(-87.9 42)POINT(-87.7 42)' b'POINT(-87.6 41.9)POINT(-87.6 41.9)'
 b'POINT(-87.7 41.9)POINT(-87.9 42)']
payment_type: [b'Cash' b'Cash' b'Cash']
pickup_grid: [b'POINT(-87.9 42)' b'POINT(-87.6 41.9)' b'POINT(-87.7 41.9)']
tip_bin: [0 0 0]
trip_day: [6 4 2]
trip_day_of_week: [5 3 1]
trip_hour: [23 22 16]
trip_miles: [13.19  3.17 29.6 ]
trip_month: [2 2 2]
trip_seconds: [1815  711 5340]



## 3. Data validation

### Import raw schema

In [15]:
from tfx.v1.dsl import Importer

In [16]:
schema_importer = Importer(
    source_uri=RAW_SCHEMA_DIR,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False
)

context.run(schema_importer)

ExecutionResult(
    component_id: Importer
    execution_id: 5
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 18
        uri: "src/raw_schema"
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.8.0"
          }
        }
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Generate statistics

In [17]:
statistics_gen = tfx.components.StatisticsGen(
    examples=train_example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 6
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [18]:
!rm -r {RAW_SCHEMA_DIR}/.ipynb_checkpoints/

rm: cannot remove 'src/raw_schema/.ipynb_checkpoints/': No such file or directory


### Validate statistics against schema

In [19]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'],
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 7
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [20]:
context.show(example_validator.outputs['anomalies'])

## 4. Data transformation

In [21]:
_transform_module_file = 'src/preprocessing/transformations.py'

transform = tfx.components.Transform(
    examples=train_example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=_transform_module_file,
    splits_config=transform_pb2.SplitsConfig(
        analyze=['train'], transform=['train', 'eval']),
)

context.run(transform, enable_cache=True)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying etl.py -> build/lib
copying transformations.py -> build/lib
installing to /tmp/tmpam23zmez
running install
running install_lib
copying build/lib/transformations.py -> /tmp/tmpam23zmez
copying build/lib/etl.py -> /tmp/tmpam23zmez
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpam23zmez/tfx_user_code_Transform-0.0+de07c8431e7a29dced215501daf4f187c64541d3189d2529c8a52c51eb6c9d4d-py3.7.egg-info
running inst

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


Processing /tmp/tmplc34xs5q/tfx_user_code_Transform-0.0+de07c8431e7a29dced215501daf4f187c64541d3189d2529c8a52c51eb6c9d4d-py3-none-any.whl

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


Processing /tmp/tmpzotfgk1m/tfx_user_code_Transform-0.0+de07c8431e7a29dced215501daf4f187c64541d3189d2529c8a52c51eb6c9d4d-py3-none-any.whl

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


Processing /tmp/tmpjlfve6z1/tfx_user_code_Transform-0.0+de07c8431e7a29dced215501daf4f187c64541d3189d2529c8a52c51eb6c9d4d-py3-none-any.whl



[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


2022-09-10 23:41:49.554081: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


ExecutionResult(
    component_id: Transform
    execution_id: 8
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

### Read sample transformed tfrecords

In [22]:
transformed_train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, "Split-train/*")
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

tft_output = tft.TFTransformOutput(transform_graph_uri)
transform_feature_spec = tft_output.transformed_feature_spec()

for input_features, target in data.get_dataset(
    transformed_train_uri, transform_feature_spec, batch_size=3).take(1):
    for key in input_features:
        print(f"{key} ({input_features[key].dtype}): {input_features[key].numpy().tolist()}")
    print(f"target: {target.numpy().tolist()}")

dropoff_grid_xf (<dtype: 'int64'>): [0, 7, 8]
euclidean_xf (<dtype: 'float32'>): [-0.6674785614013672, 1.0090183019638062, 1.7906255722045898]
loc_cross_xf (<dtype: 'int64'>): [0, 37, 79]
payment_type_xf (<dtype: 'int64'>): [0, 0, 0]
pickup_grid_xf (<dtype: 'int64'>): [0, 4, 9]
trip_day_of_week_xf (<dtype: 'int64'>): [0, 1, 1]
trip_day_xf (<dtype: 'int64'>): [1, 5, 2]
trip_hour_xf (<dtype: 'int64'>): [10, 17, 11]
trip_miles_xf (<dtype: 'float32'>): [-0.6508637070655823, 2.204909563064575, 2.957416296005249]
trip_month_xf (<dtype: 'int64'>): [0, 0, 0]
trip_seconds_xf (<dtype: 'float32'>): [-0.8767567873001099, 0.2179492861032486, 0.6437397599220276]
target: [0, 0, 0]


## 5. Model training

In [23]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.experimental import latest_artifacts_resolver
from tfx.dsl.experimental import latest_blessed_model_resolver

### Get the latest model to warm start

In [24]:
latest_model_resolver = Resolver(
    strategy_class=latest_artifacts_resolver.LatestArtifactsResolver,
    latest_model=tfx.types.Channel(type=tfx.types.standard_artifacts.Model)
)

context.run(latest_model_resolver, enable_cache=True)

ExecutionResult(
    component_id: Resolver
    execution_id: 9
    outputs:
        latest_model: OutputChannel(artifact_type=Model, producer_component_id=Resolver, output_key=latest_model, additional_properties={}, additional_custom_properties={}))

### Train the model

In [25]:
_train_module_file = 'src/model_training/runner.py'

trainer = tfx.components.Trainer(
    module_file=_train_module_file,
    examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    base_model=latest_model_resolver.outputs['latest_model'],
    transform_graph=transform.outputs['transform_graph'],
    hyperparameters=hyperparams_gen.outputs['hyperparameters'],
)

context.run(trainer, enable_cache=True)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying data.py -> build/lib
copying exporter.py -> build/lib
copying trainer.py -> build/lib
copying task.py -> build/lib
copying model.py -> build/lib
copying runner.py -> build/lib
copying defaults.py -> build/lib
installing to /tmp/tmplhuxpwtg
running install
running install_lib
copying build/lib/trainer.py -> /tmp/tmplhuxpwtg
copying build/lib/model.py -> /tmp/tmplhuxpwtg
copying build/lib/runner.py -> /tmp/tmplhuxpwtg
copying build/lib/task.py -> /tmp/tmplhuxpwtg
copying build/lib/data.py -> /tmp/tmplhuxpwtg
copying build/lib/defaults.py -> /tmp/tmplhuxpwtg
copying build/lib/exporter.py -> /tmp/tmplhuxpwtg
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manif

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


Processing /tmp/tmp_na4tyvb/tfx_user_code_Trainer-0.0+437642825399ecb8802c98273bfd8605f1af4f0d57f21bf741e670d64447bfc8-py3-none-any.whl

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


Epoch 1/5
7/7 [==============================] - 3s 305ms/step - loss: 0.8057 - accuracy: 0.9609 - val_loss: 0.6804 - val_accuracy: 1.0000
Epoch 2/5
7/7 [==============================] - 1s 108ms/step - loss: 0.6025 - accuracy: 1.0000 - val_loss: 0.5143 - val_accuracy: 1.0000
Epoch 3/5
7/7 [==============================] - 1s 127ms/step - loss: 0.4481 - accuracy: 1.0000 - val_loss: 0.3774 - val_accuracy: 1.0000
Epoch 4/5
7/7 [==============================] - 1s 114ms/step - loss: 0.3165 - accuracy: 1.0000 - val_loss: 0.2501 - val_accuracy: 1.0000
Epoch 5/5
7/7 [==============================] - 4s 694ms/step - loss: 0.2041 - accuracy: 1.0000 - val_loss: 0.1522 - val_accuracy: 1.0000


2022-09-10 23:43:29.304220: I tensorflow/core/platform/cloud/gcs_file_system.cc:581] ### RequestUploadSessionStatus: completed = 1, already_uploaded = 0, file = gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/tfx_artifacts_interactive/chicago-taxi-tips-classifier-v01-train-pipeline/Trainer/model_run/10/validation/events.out.tfevents.1662853401.python-20220624-164215-bjacob.31201.1.v2


ExecutionResult(
    component_id: Trainer
    execution_id: 10
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## 6. Model evaluation

### Get the latest blessed model for model validation.

In [26]:
blessed_model_resolver = Resolver(
    strategy_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
    model=tfx.types.Channel(type=tfx.types.standard_artifacts.Model),
    model_blessing=tfx.types.Channel(type=tfx.types.standard_artifacts.ModelBlessing)
)

context.run(blessed_model_resolver, enable_cache=True)

ExecutionResult(
    component_id: Resolver
    execution_id: 11
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

### Evaluate and validate the model against the baseline model.

In [27]:
from tfx.components import Evaluator

In [28]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name='serving_tf_example',
            label_key=features.TARGET_FEATURE_NAME,
            prediction_key='probabilities')
    ],
    slicing_specs=[
        tfma.SlicingSpec(),
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[   
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(
                    class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.8}),
                        # Change threshold will be ignored if there is no
                        # baseline model resolved from MLMD (first run).
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': -1e-10}))),
        ])
    ])


evaluator = Evaluator(
    examples=test_example_gen.outputs['examples'],
    example_splits=['test'],
    model=trainer.outputs['model'],
    baseline_model=blessed_model_resolver.outputs['model'],
    eval_config=eval_config,
    schema=schema_importer.outputs['result']
)

context.run(evaluator, enable_cache=True)

ExecutionResult(
    component_id: Evaluator
    execution_id: 12
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [29]:
evaluation_results = evaluator.outputs['evaluation'].get()[0].uri
print("validation_ok:", tfma.load_validation_result(evaluation_results).validation_ok, '\n')

for entry in list(tfma.load_metrics(evaluation_results))[0].metric_keys_and_values:
    value = entry.value.double_value.value
    if value:
        print(entry.key.name, ":", round(entry.value.double_value.value, 3))

validation_ok: True 

accuracy : 1.0
loss : 0.767
example_count : 1000.0
binary_accuracy : 1.0


## 7. Model pushing

In [30]:
exported_model_location = os.path.join(MODEL_REGISTRY, MODEL_DISPLAY_NAME)

push_destination=tfx.proto.pusher_pb2.PushDestination(
    filesystem=tfx.proto.pusher_pb2.PushDestination.Filesystem(
        base_directory=exported_model_location,
    )
)

pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=push_destination
)

context.run(pusher, enable_cache=True)

ExecutionResult(
    component_id: Pusher
    execution_id: 13
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

## 8. Model Upload to Vertex AI

In [31]:
serving_runtime = 'tf2-cpu.2-8'
serving_image_uri = f"us-docker.pkg.dev/vertex-ai/prediction/{serving_runtime}:latest"

labels = {
    'dataset_name': DATASET_DISPLAY_NAME,
    'pipeline_name': PIPELINE_NAME
}
labels = json.dumps(labels)

vertex_model_uploader = components.vertex_model_uploader(
    project=PROJECT,
    region=REGION,
    model_display_name=MODEL_DISPLAY_NAME,
    pushed_model_location=exported_model_location,
    serving_image_uri=serving_image_uri,
    model_blessing=evaluator.outputs['blessing'],
    explanation_config='',
    labels=labels
)

context.run(vertex_model_uploader, enable_cache=True)

Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/55590906972/locations/us-central1/models/5832755253723791360/operations/362430162437931008


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/55590906972/locations/us-central1/models/5832755253723791360/operations/362430162437931008


Model created. Resource name: projects/55590906972/locations/us-central1/models/5832755253723791360


...

To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/55590906972/locations/us-central1/models/5832755253723791360')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/55590906972/locations/us-central1/models/5832755253723791360')


ExecutionResult(
    component_id: vertex_model_uploader
    execution_id: 14
    outputs:
        uploaded_model: OutputChannel(artifact_type=File, producer_component_id=vertex_model_uploader, output_key=uploaded_model, additional_properties={}, additional_custom_properties={}))

In [32]:
vertex_model_uploader.outputs['uploaded_model'].get()[0].get_string_custom_property('model_uri')

'projects/55590906972/locations/us-central1/models/5832755253723791360'